# CNN Trainig Templete
## Based on Tensorlfow - CNN MNIST example
https://github.com/tensorflow/tensorflow/blob/r1.6/tensorflow/examples/tutorials/layers/cnn_mnist.py

## 필요한 library 호출

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import random
import time
from sklearn.cross_validation import train_test_split
from datetime import datetime

tf.logging.set_verbosity(tf.logging.INFO)


# Data loading

In [ ]:
width = 640
height = 480
channel = 3
total_pix = width * height * channel
num_class = 2

In [ ]:
def data_process(d_dir, split_rate):
    
    num_class = len(d_dir.keys())

    out = {}
    

    for i in range(num_class):
        x_train, x_test = train_test_split(d_dir[i], test_size=split_rate)
        
        if i == 0:
            train_temp = x_train
            test_temp = x_test
        else:
            train_temp.extend(x_train)
            test_temp.extend(x_test)
    
    random.shuffle(train_temp)
    out['train'] = train_temp
    random.shuffle(test_temp)
    out['test'] = test_temp
    
    return out

In [ ]:
image_dir1 = '/dataset/'
class_name1 = ['untorn', 'torn']

data_dir1 = {}
for i in range(len(class_name1)):
    temp_dir = image_dir1 + class_name1[i]
    img_files = [[os.path.join(temp_dir, f), np.int16(i)] for f in os.listdir(temp_dir) if f.endswith('.png')]
    data_dir1[i] = img_files
print(len(data_dir1[0]))

In [ ]:
tr_data_dir = data_process(data_dir1, 0.2)

In [ ]:
tr_data = np.zeros((len(tr_data_dir['train']), total_pix), dtype=np.float32)
tr_label = np.zeros((len(tr_data_dir['train']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['train'])):
    img = cv2.imread(tr_data_dir['train'][i][0])
    tr_data[i][:] = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC).flatten()
    tr_label[i] = tr_data_dir['train'][i][1]
    
    

    
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0])
    ev_data[i][:] = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC).flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

# CNN model

In [ ]:
X = tf.placeholder(tf.float32, shape = [None, total_pix])
Y = tf.placeholder(tf.int32, shape = [None, 1])
keep_prob = tf.placeholder(tf.float32)

In [ ]:
def cnn_model_fn(features, labels, mode):
    
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, height, width, channels]
    # Our Fishing net image size is 640x480 and 3-channel (RGB)
    input_layer = tf.reshape(features["x"], [-1, 480, 640, 3])

    # Convolutional Layer #1
    # Computes 48 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 480, 640, 3]
    # Output Tensor Shape: [batch_size, 480, 640, 48]
    conv1 = tf.layers.conv2d(
        
        inputs=input_layer,
        filters=48,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
        name = 'conv1')
    #print(conv1.shape)
    

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 480, 640, 48]
    # Output Tensor Shape: [batch_size, 240, 320, 48]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, name='pool1')
    #print(pool1.shape)

    # Convolutional Layer #2
    # Computes 96 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 240, 320, 48]
    # Output Tensor Shape: [batch_size, 240, 320, 96]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
        name = 'conv2')
    #print(conv2.shape)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 240, 320, 96]
    # Output Tensor Shape: [batch_size, 120, 160, 96]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, name='pool2')
    #print(pool2.shape)

    
    
    
    # Input Tensor Shape: [batch_size, 120, 160, 96]
    # Output Tensor Shape: [batch_size, 120, 160, 96]    
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
        name = 'conv3')
    #print(conv3.shape)
    
    # Input Tensor Shape: [batch_size, 120, 160, 96]
    # Output Tensor Shape: [batch_size, 60, 80, 96]
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2, name='pool3')
    #print(pool3.shape)
    
    
    
    # Input Tensor Shape: [batch_size, 60, 80, 96]
    # Output Tensor Shape: [batch_size, 60, 80, 96]    
    conv4 = tf.layers.conv2d(
        inputs=pool3,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
        name = 'conv4')
    #print(conv4.shape)
    
    # Input Tensor Shape: [batch_size, 60, 80, 96]
    # Output Tensor Shape: [batch_size, 30, 40, 96]
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2, name='pool4')
    #print(pool4.shape)
    
    
    
    # Input Tensor Shape: [batch_size, 30, 40, 96]
    # Output Tensor Shape: [batch_size, 30, 40, 96]    
    conv5 = tf.layers.conv2d(
        inputs=pool4,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
        name = 'conv5')
    #print(conv5.shape)
    
    # Input Tensor Shape: [batch_size, 30, 40, 96]
    # Output Tensor Shape: [batch_size, 15, 20, 96]
    pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[2, 2], strides=2, name='pool5')
    #print(pool5.shape)
    
    
    
    
    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 15, 20, 96]
    # Output Tensor Shape: [batch_size, 15 * 20 * 96]
    pool5_flat = tf.reshape(pool5, [-1, 15 * 20 * 96])
    #print(pool5_flat.shape)


    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 15 * 20 * 96]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool5_flat, units=1024, activation=tf.nn.relu, name='fc1')
    #print(dense.shape)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN, name='fc_drop')

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 1]
    logits = tf.layers.dense(inputs=dropout, units=2, name='logits')
    #print(logits.shape)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1, name='argmax'),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    # sparse_softmax_cross_entropy cannot use one-hot encoding
    
    #loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# Training

In [ ]:
today = datetime.today().strftime("%m-%d-%H-%M")
print(today)

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
session = tf.Session(config=config)

# Create the Estimator
net_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/models/CNN2600_vis_%s" % today)

# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

batch_s = 16
epochs = 45
tr_steps = int(tr_data.shape[0]/batch_s*epochs)
in_steps = 1
print(tr_steps)



# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": tr_data},
    y=tr_label,
    batch_size=batch_s,
    num_epochs=epochs,
    shuffle=True)
net_classifier.train(
    input_fn=train_input_fn,
    steps=tr_steps,
    hooks=[logging_hook])

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = tf.Session(config=config)

ev_results = np.zeros((len(ev_label), 1))


# Evaluate the model and print results
for i in range(len(ev_label)):
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": ev_data[i:i+1]},
        y=ev_label[i:i+1],
        num_epochs=1,
        shuffle=False)
    eval_results = net_classifier.evaluate(input_fn=eval_input_fn)

    ev_results[i] = eval_results["accuracy"]

In [ ]:
print(np.mean(ev_results))

In [ ]:
net_classifier.get_variable_names()

In [ ]:
net_classifier.get_variable_value('logits/kernel')

In [ ]:
weights = net_classifier.get_variable_value('conv1/kernel')
print(weights.shape)

In [ ]:
test = weights[:,:,:,47]

In [ ]:
print(test.shape)
print(np.max(test))

In [ ]:
plt.imshow(weights, cmap='gray')
plt.show()

In [ ]:
fail_idx = np.where(ev_results == 0)
print(fail_idx[0])
print(len(fail_idx[0]))